In [ ]:
"""
### CODE 1 ####
ECI constituency start/end mapper + row counter
- Finds each "Constituency <no> <name>" (start)
- Finds each "TURN OUT"/"Turnout" (end)
- Pairs every TURN OUT with the most recent unmatched Constituency (stack logic)
- Extracts number_of_rows = last serial number in the candidate list just before TURN OUT
- Saves JSON with: constituency_no, constituency_name, start_page, end_page, number_of_rows
Edit ELECTIONS_PDF and CONSTITUENCY_MAP_JSON and run.

### CODE 2 ####
Extraction of the data contained in the Election PDF 
- Creates PDF chunk of each page in the PDF
- Sends them to Gemini ; saves them as JSON chunks
- If constituency_no stops or not well extracted, delete the json chunk and re-run CODE 2
- Code 2 is meant to be re-run and use past chunks

### CODE 3 ####
- Cleans the CSV file and rename columns to TCPD standards
- creates N_Cand, n_rows, margin, margin_percentage, etc.

### CODE 4 ####
- Compare "CONSTITUENCY_MAP_JSON" and "CLEANED_ELECTION_CSV" on :
    - number of rows
    - page_number_start
    - page_number_end
    
If one constituency has not been correctly exctracted
    & delete its chunk file and re-run CODE 2


# COST & EFFICIENCY
- We tried to make this code as cost efficient as possible
- For 372 pages, the Gemini cost us 17 euros (1700 Rs.)
- This cost appears "pretty expensive"

"""
import os
import re
import json
import logging
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import pdfplumber  # pip install pdfplumber

# --- Configure paths ---
BASE_DIR = os.path.expanduser("~/Desktop/ECI Statistical Reports")
ELECTION_PDF_DIR = os.path.join(BASE_DIR, "Statistical Reports PDF")
ELECTIONS_PDF = os.path.join(ELECTION_PDF_DIR, "10-Detailed_Results_1744893339.pdf")
NO_OF_CONSTITUENCY = 288 # Number of constituencies in the state
PDF_TOTAL_PAGES = 372  # Set to full run based on your input (all N pages will be processed if no chunk exist)

# --- Configuration ---
YOUR_API_KEY = "YOUR_API_KEY"  # 🛑 Replace with your actual key

# Information about the State
DELIMID = 4
ELECTION_TYPE = "State Assembly Election (AE)" # "Lok Sabha Election (GE)"
SABHA_TYPE = "AE" # AE or GE 
STATE_NAME = "Maharashtra"
ASSEMBLY_NO = 14
YEAR = 2024
MONTH = 11
POLL_NO = 0

# Automatically Create Files name with State_Name + Year in their name.
STATE_YEAR_PREFIX = f"{STATE_NAME.replace(' ', '_')}_{SABHA_TYPE}_{YEAR}"
ELECTION_DIR = os.path.join(BASE_DIR, f"{STATE_YEAR_PREFIX}")
CONSTITUENCY_MAP_JSON = os.path.join(ELECTION_DIR, f"{STATE_YEAR_PREFIX}_constituency_start_end_map.json")
TEMP_CACHE_DIR = os.path.join(ELECTION_DIR, f"{STATE_YEAR_PREFIX}_temp_json_cache_full_run")
ELECTION_JSON = os.path.join(ELECTION_DIR, f"{STATE_YEAR_PREFIX}_elections.json")
ELECTION_CSV = os.path.join(ELECTION_DIR, f"{STATE_YEAR_PREFIX}_elections.csv")
CLEANED_ELECTION_CSV = ELECTION_CSV #Change this in case you want an unclean and a cleaned final csv

# ---- Silence pdfminer/pdfplumber chatter (e.g., "CropBox missing...") ----
for name in ("pdfminer", "pdfminer.layout", "pdfminer.pdfpage", "pdfplumber"):
    logging.getLogger(name).setLevel(logging.ERROR)
    lg = logging.getLogger(name)
    lg.propagate = False
    if lg.handlers:
        lg.handlers.clear()

# ---- Regex patterns ----
# Constituency header examples:
#   "Constituency 187 - COLABA ( TOTAL ELECTORS - 265326 )"
#   "Constituency 188 PANVEL ( TOTAL ELECTORS - 652291 )"
CONST_RE = re.compile(
    r'\bConstituency\s+(\d+)\s*(?:[-–—]\s*)?([^\n\r(]+)?',
    flags=re.IGNORECASE
)

# TURN OUT appears as "TURN OUT" or "Turnout"
TURNOUT_RE = re.compile(r'\bTURN\s*OUT\b|\bTURNOUT\b', flags=re.IGNORECASE)

# Candidate line number: line starting with integer + space (but NOT a header like "1 - AKKALKUWA")
CAND_NUM_RE = re.compile(r'(?m)^\s*(\d{1,3})\s+(?!-)')

def clean_const_name(name_raw: Optional[str]) -> Optional[str]:
    if not name_raw:
        return None
    name = name_raw.strip()
    name = re.sub(r'\s{2,}', ' ', name).strip(" -\t\r\n")
    name = re.sub(r'\s*\(.*$', '', name)                # drop "( TOTAL ELECTORS - ... )"
    name = re.sub(r'\s*TOTAL.*$', '', name, flags=re.IGNORECASE)
    return name.strip() or None

def extract_page_texts(pdf_path: str) -> List[str]:
    """Return list of page texts, 1-based index conceptually (we'll store in a 0-based list)."""
    texts: List[str] = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    return texts

def find_events(page_texts: List[str]) -> List[Tuple[int, int, str, Dict[str, Any]]]:
    """
    Return list of events as tuples:
      (page_index_1based, offset_in_page_text, 'start'|'turnout', payload)
    payload for 'start': {'no': int, 'name': str|None}
    payload for 'turnout': {}
    """
    events: List[Tuple[int, int, str, Dict[str, Any]]] = []
    for pidx_0, text in enumerate(page_texts):
        pidx = pidx_0 + 1
        for m in CONST_RE.finditer(text):
            events.append((pidx, m.start(), "start", {
                "no": int(m.group(1)),
                "name": clean_const_name(m.group(2))
            }))
        for m in TURNOUT_RE.finditer(text):
            events.append((pidx, m.start(), "turnout", {}))
    events.sort(key=lambda e: (e[0], e[1]))
    return events

def pair_constituencies(events: List[Tuple[int, int, str, Dict[str, Any]]]) -> List[Dict[str, Any]]:
    """
    Pair each TURN OUT to the most recent unmatched Constituency.
    Also keep start/end offsets so we can slice text precisely to count rows.
    """
    mapping: Dict[int, Dict[str, Any]] = {}
    open_stack: List[Tuple[int, int, int]] = []  # list of (const_no, start_page, start_offset)

    for page, offset, evtype, payload in events:
        if evtype == "start":
            cno = int(payload["no"])
            cname = payload["name"]
            if cno not in mapping:
                mapping[cno] = {
                    "constituency_no": cno,
                    "constituency_name": cname,
                    "page_number_start": page,
                    "start_offset": offset,
                    "page_number_end": None,
                    "end_offset": None,
                    "number_of_rows": None
                }
            else:
                # keep earliest start; backfill name if missing
                if mapping[cno]["constituency_name"] is None and cname:
                    mapping[cno]["constituency_name"] = cname
                if page < mapping[cno]["page_number_start"]:
                    mapping[cno]["page_number_start"] = page
                    mapping[cno]["start_offset"] = offset
            open_stack.append((cno, page, offset))

        elif evtype == "turnout":
            # Close the latest unmatched constituency
            while open_stack:
                cno, sp, so = open_stack.pop()
                if mapping[cno]["page_number_end"] is None:
                    mapping[cno]["page_number_end"] = page
                    mapping[cno]["end_offset"] = offset
                    break
            # else: stray TURN OUT; ignore

    # sorted list by constituency number
    return sorted(mapping.values(), key=lambda d: d["constituency_no"])

def slice_constituency_text(entry: Dict[str, Any], page_texts: List[str]) -> str:
    """
    Concatenate the text for a constituency between its start and TURN OUT markers (inclusive bounds trimmed).
    """
    sp, so = entry["page_number_start"], entry["start_offset"]
    ep, eo = entry["page_number_end"], entry["end_offset"]
    if sp is None or ep is None:
        return ""

    # Convert to 0-based indexes for our list
    sp0, ep0 = sp - 1, ep - 1

    parts: List[str] = []
    if sp == ep:
        segment = page_texts[sp0][so:eo]
        parts.append(segment)
    else:
        # first page: from start_offset to end of page
        parts.append(page_texts[sp0][so:])
        # middle pages: whole text
        for p in range(sp0 + 1, ep0):
            parts.append(page_texts[p])
        # end page: from start of page to end_offset
        parts.append(page_texts[ep0][:eo])

    return "\n".join(parts)

def count_rows_in_segment(segment: str) -> Optional[int]:
    """
    Return the largest leading integer on lines within the segment
    (i.e., the last candidate serial number before TURN OUT).
    """
    if not segment:
        return None
    nums = [int(m.group(1)) for m in CAND_NUM_RE.finditer(segment)]
    return max(nums) if nums else None

def build_mapping_with_rows(pdf_path: str) -> List[Dict[str, Any]]:
    page_texts = extract_page_texts(pdf_path)
    events = find_events(page_texts)
    mapping = pair_constituencies(events)

    # Sanity quick stats
    total = len(mapping)
    missing_end = sum(1 for m in mapping if m["page_number_end"] is None)
    print(f"[Sanity] constituencies mapped: {total} | missing end: {missing_end}")

    # Compute number_of_rows for each completed constituency
    for m in mapping:
        if m["page_number_end"] is not None and m["page_number_start"] is not None:
            seg = slice_constituency_text(m, page_texts)
            m["number_of_rows"] = count_rows_in_segment(seg)

    # Remove offsets from final output (keep page_number_start/page_number_end + number_of_rows)
    for m in mapping:
        m.pop("start_offset", None)
        m.pop("end_offset", None)

    return mapping

def main():
    mapping = build_mapping_with_rows(ELECTIONS_PDF)

    # ✅ Ensure output directory exists
    os.makedirs(ELECTION_DIR, exist_ok=True)

    Path(CONSTITUENCY_MAP_JSON).write_text(json.dumps(mapping, indent=2, ensure_ascii=False), encoding="utf-8")
    print(f"Saved mapping → {CONSTITUENCY_MAP_JSON}")

    # Print requested constituency
    tgt = next((m for m in mapping if m["constituency_no"] == NO_OF_CONSTITUENCY), None)
    if tgt:
        print(f"\nConstituency {tgt['constituency_no']} — {tgt['constituency_name'] or '(name unknown)'}")
        print(f"  page_number_start: {tgt['page_number_start']}")
        print(f"  page_number_end:   {tgt['page_number_end']}")
        print(f"  number_of_rows:    {tgt['number_of_rows']}")
    else:
        print(f"\nConstituency {NO_OF_CONSTITUENCY} not found.")

if __name__ == "__main__":
    main()

In [ ]:
### CODE 2 ####
# - Extract PDF chunks of each Page 
# - Build chunks json into a final JSON & CSV file for clean ECI results

import os
import json
import csv
import pypdf
import google.generativeai as genai
from google.api_core import exceptions
import time
import shutil
import re  # Import regex for splitting constituency name

# --- Parameters ---
CHUNK_SIZE = 1                            # 1 page per chunk (Most reliable)
MODEL_NAME = 'models/gemini-2.5-pro'     # The reliable model

# --- JSON Schema (Kept simple for extraction, transformation happens in Python) ---
JSON_SCHEMA = """
[
  {
    "page_number": "integer (The page number of the PDF this data was extracted from)",
    "constituency_name": "string (e.g., '1 - AKKALKUWA (ST)')",
    "total_electors": "integer or null (Extract from 'TOTAL ELECTORS' in the constituency header)",
    "turnout_total_votes": "integer or null (Extract the 'TOTAL' vote count from the 'TURN OUT' row)",
    "turnout_percentage": "float or null (Extract the '% VOTES POLLED' from the 'TURN OUT' row)",
    "page_number_end": "integer or null (the page where the constituency ends; null if it continues)",
    "candidates": [
      {
        "rank": "integer",
        "name": "string (MUST include NOTA as a candidate named 'NOTA')",
        "gender": "string (MALE/FEMALE/NULL for NOTA)",
        "age": "integer or null",
        "category": "string or null",
        "party": "string (e.g., SHS or NOTA)",
        "symbol": "string",
        "general_votes": "integer",
        "postal_votes": "integer",
        "total_votes": "integer",
        "percent_valid_votes": "float",
        "percent_total_electors": "float"
      }
    ]
  }
]
"""

# --- Core Functions ---

def get_pdf_page_count(input_path):
    """Safely determines the total number of pages in the PDF."""
    try:
        reader = pypdf.PdfReader(input_path)
        return len(reader.pages)
    except Exception:
        print(f"ERROR: The input PDF '{input_path}' was not found or is corrupted.")
        return 0

def create_pdf_chunk(input_path, output_path, start_page, end_page):
    """Creates a new PDF with only the specified page range (1-indexed)."""
    reader = pypdf.PdfReader(input_path)
    writer = pypdf.PdfWriter()

    for page_num in range(start_page - 1, end_page):
        writer.add_page(reader.pages[page_num])

    with open(output_path, "wb") as f:
        writer.write(f)

def clean_json_response(raw_response):
    """Cleans the AI's response to extract the JSON list."""
    try:
        start_index = raw_response.find('[')
        end_index = raw_response.rfind(']')
        if start_index != -1 and end_index != -1 and end_index > start_index:
            json_str = raw_response[start_index: end_index + 1]
            return json.loads(json_str)
        else:
            return []
    except json.JSONDecodeError:
        return []

def extract_data_from_chunk(pdf_chunk_path, last_constituency_context, page_num):
    """Uploads a PDF chunk and extracts data using the specified Gemini model."""
    print(f"\n--- Processing Chunk for Page {page_num}: {pdf_chunk_path} ---")
    if last_constituency_context:
        print(f"--- Context: Continuing constituency '{last_constituency_context}' ---")

    model = genai.GenerativeModel(MODEL_NAME)
    max_retries = 3
    backoff_seconds = 5

    for attempt in range(1, max_retries + 1):
        uploaded_file = None
        try:
            uploaded_file = genai.upload_file(path=pdf_chunk_path, display_name=pdf_chunk_path)

            prompt = f"""
            You are a data extraction bot. Extract all election data from the provided PDF chunk (page {page_num} of the full PDF).
            Your output MUST be a valid JSON list adhering to this exact schema:
            {JSON_SCHEMA}

            INSTRUCTIONS:
            1. For every constituency detected in this chunk, you MUST include a 'page_number' field with value {page_num}.
            2. Constituency data can span multiple pages. If this page continues a constituency from the previous page, return all extracted candidates/data for it.
            3. The constituency boundary is marked by the 'Constituency' header (e.g. 'Constituency 1 - AKKALKUWA (ST) ( TOTAL ELECTORS - ... )') and ends immediately after the 'TURN OUT TOTAL:' row for that constituency.
            4. You MUST extract 'constituency_name' exactly as it appears in the header, but WITHOUT the word 'Constituency'. For example, from 'Constituency 1 - AKKALKUWA (ST) ( TOTAL ELECTORS - 319481)' you must use:
               "constituency_name": "1 - AKKALKUWA (ST)".
            5. You MUST extract:
               - 'total_electors' from the '( TOTAL ELECTORS - ... )' part of the header,
               - 'turnout_total_votes' (the TOTAL column) and
               - 'turnout_percentage' (the '% VOTES POLLED' value)
               from the 'TURN OUT TOTAL:' row whenever that row is present on this page.
            6. You MUST treat **NOTA** as a regular candidate and include it in the 'candidates' list.
            7. For each constituency, you MUST extract all candidate rows in correct order, up to (and not beyond) the 'TURN OUT TOTAL:' row for that constituency. The extraction for a given constituency is only complete once the 'TURN OUT TOTAL:' row has been seen.
            8. You MUST include a 'page_number_end' field for each constituency object:
               - If this page contains the 'TURN OUT TOTAL:' row for that constituency, set 'page_number_end' to {page_num}.
               - If the 'TURN OUT TOTAL:' row is NOT present for that constituency in this chunk (i.e. the candidate list continues on the next page), set 'page_number_end' to null for that constituency in this page.
            9. If this page starts in the middle of a constituency (continuation from a previous page), do NOT repeat candidates that already appeared on previous pages. Only add the remaining candidate rows visible on this page, followed by the 'TURN OUT TOTAL:' row if it appears here.
            10. Your final answer MUST be a single valid JSON list matching the schema exactly.
            """

            if last_constituency_context:
                prompt += f"""
            CONTEXT: The previous page ended during the candidate list for constituency '{last_constituency_context}'.
            Any data at the very beginning of this new chunk belongs to '{last_constituency_context}' until a new 'Constituency' header appears.
            """

            response = model.generate_content([prompt, uploaded_file])

            # Cleanup upload
            try:
                genai.delete_file(uploaded_file.name)
            except Exception:
                pass

            extracted_data = clean_json_response(response.text)

            new_context = None
            if extracted_data:
                last_entry = extracted_data[-1]
                # If the last constituency on this page has no TURN OUT data yet or
                # explicit page_number_end is null, we assume it continues.
                if (last_entry.get("turnout_total_votes") is None or
                        last_entry.get("page_number_end") is None):
                    new_context = last_entry.get("constituency_name")
                    if new_context:
                        print(f"INFO: Page {page_num} ends during constituency: {new_context}")

            return extracted_data, new_context

        except Exception as e:
            print(f"ERROR: API call failed for page {page_num}, attempt {attempt}/{max_retries}: {e}")
            # Cleanup upload if something went wrong
            if uploaded_file is not None:
                try:
                    genai.delete_file(uploaded_file.name)
                except Exception:
                    pass

            if attempt < max_retries:
                print(f"Retrying page {page_num} after {backoff_seconds} seconds...")
                time.sleep(backoff_seconds)
            else:
                print(f"Giving up on page {page_num} after {max_retries} failed attempts.")
                # Return empty data but keep last_context so the loop can continue
                return [], last_constituency_context

# --- Constituency Splitting Logic ---

def split_constituency_name(full_name):
    """
    Splits a constituency name string (e.g., '1 - AKKALKUWA (ST)') into three components.
    """
    constituency_no = None
    constituency_name_only = full_name
    constituency_type = "GENERAL"

    match = re.match(r'(\d+)\s*-\s*(.*?)\s*(\((ST|SC)\))?$', full_name.strip())

    if match:
        constituency_no = int(match.group(1))

        # Capture constituency name without (ST/SC)
        name_part = match.group(2).strip()
        constituency_name_only = name_part.replace(' (ST)', '').replace(' (SC)', '').strip()

        # Check for type
        type_part = match.group(4)
        if type_part in ['ST', 'SC']:
            constituency_type = type_part

    return constituency_no, constituency_name_only, constituency_type

# --- Consolidation Function ---

def consolidate_data(raw_data):
    """
    Consolidates fragmented data and adds new split constituency fields,
    including page_number_start and page_number_end.
    """
    if not raw_data:
        return []

    # Pass 1: Collect all available metadata and candidates grouped by constituency name
    constituency_map = {}

    for item in raw_data:
        name = item.get("constituency_name")
        if not name:
            continue

        if name not in constituency_map:
            constituency_map[name] = {
                "records": [],
                "total_electors": None,
                "turnout_total_votes": None,
                "turnout_percentage": None,
                "page_number_start": None,
                "page_number_end": None,
            }

        group = constituency_map[name]
        group["records"].append(item)

        # Collect the best data for fixed fields
        if item.get("total_electors") is not None:
            group["total_electors"] = item["total_electors"]
        if item.get("turnout_total_votes") is not None:
            group["turnout_total_votes"] = item["turnout_total_votes"]
        if item.get("turnout_percentage") is not None:
            group["turnout_percentage"] = item["turnout_percentage"]

        # Track start page (minimum page_number)
        pn = item.get("page_number")
        if pn is not None:
            if group["page_number_start"] is None or pn < group["page_number_start"]:
                group["page_number_start"] = pn

        # Track end page (maximum non-null page_number_end)
        pne = item.get("page_number_end")
        if pne is not None:
            if group["page_number_end"] is None or pne > group["page_number_end"]:
                group["page_number_end"] = pne

    # Pass 2: Re-assemble the final consolidated list and apply constituency split
    final_data = []

    # Sort constituencies by the page they first appear on
    def _start_key(item):
        data = item[1]
        if data["page_number_start"] is not None:
            return data["page_number_start"]
        # Fallback if for some reason page_number_start is missing
        return data["records"][0].get("page_number", 0)

    sorted_constituencies = sorted(constituency_map.items(), key=_start_key)

    for name, data in sorted_constituencies:
        all_candidates = []
        for record in data["records"]:
            all_candidates.extend(record.get("candidates", []))

        # Apply the splitting logic (number / name / type)
        constituency_no, constituency_name_only, constituency_type = split_constituency_name(name)

        # Fallbacks if start/end pages somehow missing
        page_start = data["page_number_start"] or data["records"][0].get("page_number")
        page_end = data["page_number_end"] or data["records"][-1].get("page_number")

        # Create one consolidated record
        consolidated_record = {
            "page_number_start": page_start,
            "page_number_end": page_end,
            "constituency_full_name": name,       # Original for reference
            "constituency_no": constituency_no,
            "constituency_name": constituency_name_only,  # Cleaned name
            "constituency_type": constituency_type,
            "total_electors": data["total_electors"],
            "turnout_total_votes": data["turnout_total_votes"],
            "turnout_percentage": data["turnout_percentage"],
            "candidates": all_candidates,
        }

        final_data.append(consolidated_record)

    return final_data

# --- CSV Saving Function ---
def save_csv(data, filename):
    """
    Flattens the consolidated JSON data, applies renames, and saves it to a CSV file.
    """

    # Define NEW fieldnames with updated names and order
    fieldnames = [
        'constituency_no',
        'constituency_name',
        'constituency_type',
        'total_electors',
        'turnout_total_votes',
        'turnout_percentage',
        'position',        # Renamed from 'rank'
        'candidate',       # Renamed from 'name'
        'gender',
        'age',
        'candidate_type',  # Renamed from 'category'
        'party',
        'symbol',
        'general_votes',
        'postal_votes',
        'total_votes',
        'percent_valid_votes',
        'percent_total_electors',
        'constituency_full_name',  # Original for debug/reference
        'page_number_start',
        'page_number_end'          # ✅ NEW
    ]

    rows = []

    for constituency in data:
        base_data = {
            'constituency_no': constituency.get('constituency_no'),
            'constituency_name': constituency.get('constituency_name'),
            'constituency_type': constituency.get('constituency_type'),
            'constituency_full_name': constituency.get('constituency_full_name'),
            'page_number_start': constituency.get('page_number_start'),
            'page_number_end': constituency.get('page_number_end'),  # ✅ NEW
            'total_electors': constituency.get('total_electors'),
            'turnout_total_votes': constituency.get('turnout_total_votes'),
            'turnout_percentage': constituency.get('turnout_percentage')
        }

        for candidate in constituency.get('candidates', []):
            transformed_candidate = {
                'position': candidate.get('rank'),
                'candidate': candidate.get('name'),
                'gender': candidate.get('gender'),
                'age': candidate.get('age'),
                'candidate_type': candidate.get('category'),
                'party': candidate.get('party'),
                'symbol': candidate.get('symbol'),
                'general_votes': candidate.get('general_votes'),
                'postal_votes': candidate.get('postal_votes'),
                'total_votes': candidate.get('total_votes'),
                'percent_valid_votes': candidate.get('percent_valid_votes'),
                'percent_total_electors': candidate.get('percent_total_electors')
            }
            row = {**base_data, **transformed_candidate}
            rows.append(row)

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(rows)

    print(f"✅ Data successfully flattened, transformed, and saved to CSV file: {filename}")

# --- Resume Helpers ---

def get_last_context_before_page(start_page):
    """
    For a missing segment starting at start_page, look at the previous page
    (start_page - 1) in the cache and decide if we are in the middle of a constituency.
    Returns the constituency_name if the previous page ended mid-constituency, else None.
    """
    prev_page = start_page - 1
    if prev_page < 1:
        return None

    cache_filename = os.path.join(TEMP_CACHE_DIR, f"chunk_{prev_page}_data.json")
    if not os.path.exists(cache_filename):
        print(f"WARNING: No cache file found for previous page {prev_page}. No context.")
        return None

    try:
        with open(cache_filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"WARNING: Could not read cache file for page {prev_page}: {e}")
        return None

    if not data:
        return None

    last_entry = data[-1]
    cname = last_entry.get("constituency_name")

    # Continuation if turnout_total_votes is None OR page_number_end is None
    if cname and (last_entry.get("turnout_total_votes") is None or
                  last_entry.get("page_number_end") is None):
        print(f"Resumed context for page {start_page} from cached page {prev_page}: '{cname}'")
        return cname

    return None

def detect_resume_state(pages_to_process):
    """
    Determine which pages are missing from cache and need to be (re-)extracted.

    Returns:
        missing_segments: list of (start_page, end_page) tuples (inclusive)
        segment_contexts: dict {start_page: last_constituency_context_before_segment}
    """
    if not os.path.exists(TEMP_CACHE_DIR):
        os.makedirs(TEMP_CACHE_DIR)
        print("No existing cache. All pages considered missing.")
        # Entire range is missing; no context before page 1
        return [(1, pages_to_process)], {1: None}

    cache_files = [f for f in os.listdir(TEMP_CACHE_DIR)
                   if f.startswith("chunk_") and f.endswith("_data.json")]
    if not cache_files:
        print("Cache directory is empty. All pages considered missing.")
        return [(1, pages_to_process)], {1: None}

    # Extract page numbers from file names like "chunk_60_data.json"
    try:
        processed_pages = sorted(int(f.split('_')[1]) for f in cache_files)
    except Exception as e:
        print(f"WARNING: Could not parse page numbers from cache files: {e}")
        print("Assuming cache unusable. All pages considered missing.")
        return [(1, pages_to_process)], {1: None}

    pages_set = set(processed_pages)

    # Find missing pages in range 1..pages_to_process
    missing_pages = [p for p in range(1, pages_to_process + 1) if p not in pages_set]

    if not missing_pages:
        print(f"All {pages_to_process} pages already present in cache (1..{pages_to_process}). No extraction needed.")
        return [], {}

    # Group missing pages into contiguous segments: e.g. [1,2,3, 10,11] -> [(1,3), (10,11)]
    missing_segments = []
    seg_start = missing_pages[0]
    prev_p = seg_start

    for p in missing_pages[1:]:
        if p == prev_p + 1:
            prev_p = p
        else:
            missing_segments.append((seg_start, prev_p))
            seg_start = p
            prev_p = p
    missing_segments.append((seg_start, prev_p))

    # For each segment, compute context from previous page
    segment_contexts = {}
    for start_page, end_page in missing_segments:
        ctx = get_last_context_before_page(start_page)
        segment_contexts[start_page] = ctx

    print(f"Missing page segments to (re-)extract: {missing_segments}")
    return missing_segments, segment_contexts

# --- Aggregation Helper ---

def load_and_consolidate_from_cache():
    """Load all chunk JSON files from cache, consolidate, and return final data."""
    raw_aggregated_data = []
    if not os.path.exists(TEMP_CACHE_DIR):
        print("No cache directory found. Nothing to aggregate.")
        return []

    cache_files = [f for f in os.listdir(TEMP_CACHE_DIR) if f.endswith(".json")]
    if not cache_files:
        print("Cache directory is empty. Nothing to aggregate.")
        return []

    # Aggregate raw data from disk cache
    for filename in sorted(cache_files, key=lambda x: int(x.split('_')[1])):
        full_path = os.path.join(TEMP_CACHE_DIR, filename)
        try:
            with open(full_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                raw_aggregated_data.extend(data)
        except Exception as e:
            print(f"WARNING: Could not load data from {filename}: {e}")

    # Consolidate and Transform the data
    final_consolidated_data = consolidate_data(raw_aggregated_data)
    return final_consolidated_data

# --- Main Execution Loop with Resumable Caching ---

def main():
    """Runs the full extraction with 1-page chunks and disk caching, resumable."""

    # 1. Initialization and Setup
    try:
        genai.configure(api_key=YOUR_API_KEY)
    except Exception:
        print("ERROR: Could not configure Google GenAI client. Please ensure the API key and library are correct.")
        return

    total_pdf_pages = get_pdf_page_count(ELECTIONS_PDF)
    if total_pdf_pages == 0:
        return

    # Use the full page count specified
    pages_to_process = min(PDF_TOTAL_PAGES, total_pdf_pages)

    # Detect which pages are missing and need (re-)extraction
    missing_segments, segment_contexts = detect_resume_state(pages_to_process)

    temp_pdf_files = []

    if not missing_segments:
        print(f"--- No pages left to extract (all 1..{pages_to_process} already cached). Skipping extraction. ---")
    else:
        print(f"--- Will (re-)extract only missing pages, segments: {missing_segments} ---")

    # 2. Extraction Loop (only missing pages)
    try:
        for (start_page, end_page) in missing_segments:
            last_context = segment_contexts.get(start_page)
            print(f"\n=== Processing missing segment {start_page}–{end_page}, initial context: {last_context} ===")

            for page_num in range(start_page, end_page + 1):
                chunk_filename = f"temp_chunk_{page_num}.pdf"
                temp_pdf_files.append(chunk_filename)

                # Create 1-page chunk
                create_pdf_chunk(ELECTIONS_PDF, chunk_filename, page_num, page_num)

                # Extract Data
                chunk_data, last_context = extract_data_from_chunk(chunk_filename, last_context, page_num)

                # Save to Disk Cache (raw, fragmented data)
                if chunk_data:
                    cache_filename = os.path.join(TEMP_CACHE_DIR, f"chunk_{page_num}_data.json")
                    # Ensure page_number is set correctly
                    for item in chunk_data:
                        item['page_number'] = page_num
                    with open(cache_filename, 'w', encoding='utf-8') as f:
                        json.dump(chunk_data, f, indent=2)
                    print(f"INFO: Data for page {page_num} saved to disk cache.")

                # Small throttle to avoid overwhelming the API / network
                time.sleep(0.5)
                print("-" * 20)

    except Exception as e:
        print(f"\nFATAL ERROR during processing loop: {e}")

    # --- Final Steps: Aggregation and Saving ---
    print("\n--- Aggregating, Consolidating, and Transforming Data from Cache ---")

    final_consolidated_data = load_and_consolidate_from_cache()

    if final_consolidated_data:
        # Save final aggregated JSON
        with open(ELECTION_JSON, 'w', encoding='utf-8') as f:
            json.dump(final_consolidated_data, f, indent=2)
        print(f"✅ Final consolidated data saved to JSON: {ELECTION_JSON}")

        # Save the final transformed CSV
        save_csv(final_consolidated_data, ELECTION_CSV)
    else:
        print("WARNING: No data was extracted or consolidated.")

    # --- Cleanup ---
    print("\nCleaning up temporary PDF chunk files created in THIS run...")
    for filename in temp_pdf_files:
        if os.path.exists(filename):
            os.remove(filename)

    # IMPORTANT: We keep TEMP_CACHE_DIR to allow future resumes / re-aggregation.
    print(f"INFO: Keeping data cache in {TEMP_CACHE_DIR} for potential resume/debug.")
    print("Full extraction pipeline (resumable) finalized.")

if __name__ == "__main__":
    main()

In [ ]:
### CODE 3 ####
# - Clean the csv to TCPD column names standards
# Create N_Cand, margin, margin_percentage

import os
import csv
import pandas as pd

# =====================================================
# 0. LOAD CSV
# =====================================================
print("Step 1: Loading existing CSV...")
df = pd.read_csv(ELECTION_CSV)

# =====================================================
# 1. METADATA COLUMNS (SAME VALUE FOR ALL ROWS)
# =====================================================
print("Step 2: Updating metadata columns...")

df["delimid"] = DELIMID
df["election_type"] = ELECTION_TYPE
df["state_name"] = STATE_NAME
df["assembly_no"] = ASSEMBLY_NO
df["year"] = YEAR
df["month"] = MONTH
df["poll_no"] = POLL_NO

# =====================================================
# 2. SIMPLE RENAMES (DO NOT DROP ORIGINAL COLUMNS)
# =====================================================
print("Step 3: Simple renaming / copies...")

rename_map = {
    "total_electors": "electors",
    "turnout_total_votes": "valid_votes",
    "candidate_name": "candidate",
    "gender": "sex",
    "total_votes": "votes",
    "percent_valid_votes": "vote_share_percentage",
    "percent_total_electors": "electors_share_percentage"
}

# --- create a case-insensitive lookup for the current df columns ---
lower_cols = {c.lower(): c for c in df.columns}

# --- STEP 1: if both src and tgt exist (case-insensitive), merge values then drop src ---
for src_lower, tgt in rename_map.items():
    if src_lower in lower_cols and tgt.lower() in lower_cols:
        src_col = lower_cols[src_lower]
        tgt_col = lower_cols[tgt.lower()]
        df[tgt_col] = df[tgt_col].fillna(df[src_col])
        df.drop(columns=[src_col], inplace=True)
        # remove from map since it's already handled
        lower_cols.pop(src_lower, None)

# rebuild lookup because df.columns changed
lower_cols = {c.lower(): c for c in df.columns}

# --- STEP 2: rename remaining src → tgt (case-insensitive) ---
rename_dict = {}
for src_lower, tgt in rename_map.items():
    if src_lower in lower_cols:
        rename_dict[lower_cols[src_lower]] = tgt

df.rename(columns=rename_dict, inplace=True)

# =====================================================
# 3. CLEAN NUMERIC COLUMNS
# =====================================================
print("Step 4: Cleaning numeric columns...")

numeric_cols = ["votes", "valid_votes", "age", "position", "constituency_no", "assembly_no"]
for col in numeric_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.strip()
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

# =====================================================
# 4. N_Cand (PER CONST, EXCLUDING NOTA)
# =====================================================
print("Step 5: Calculating N_Cand (excluding NOTA)...")

# Group at constituency level only
group_keys_ncand = ["constituency_no"]
group_keys_ncand = [c for c in group_keys_ncand if c in df.columns]

if group_keys_ncand:
    # Build NOTA mask
    nota_mask = pd.Series(False, index=df.index)

    if "candidate" in df.columns:
        nota_mask |= (
            df["candidate"]
            .astype(str)
            .str.strip()
            .str.upper()
            .eq("NOTA")
        )

    if "party" in df.columns:
        nota_mask |= (
            df["party"]
            .astype(str)
            .str.strip()
            .str.upper()
            .eq("NOTA")
        )

    # Keep only real candidates (exclude NOTA)
    df_real = df[~nota_mask].copy()

    # Count real candidates per constituency
    ncand_counts = (
        df_real
        .groupby(group_keys_ncand)
        .size()
        .reset_index(name="n_cand")
    )

    # Merge counts back into full dataframe (NOTA rows included but not counted)
    df = df.merge(ncand_counts, on=group_keys_ncand, how="left")

    # Fill missing with 0
    df["n_cand"] = df["n_cand"].fillna(0).astype(int)

else:
    # If grouping keys missing, create N_Cand=0 column
    if "n_cand" not in df.columns:
        df["n_cand"] = 0

group_keys_nrows = ["constituency_no"]
group_keys_nrows = [c for c in group_keys_nrows if c in df.columns]

if group_keys_nrows:
    # Count ALL rows per constituency (including NOTA)
    nrows_counts = (
        df
        .groupby(group_keys_nrows)
        .size()
        .reset_index(name="number_of_rows")
    )

    df = df.merge(nrows_counts, on=group_keys_nrows, how="left")
    df["number_of_rows"] = df["number_of_rows"].fillna(0).astype(int)
    
# =====================================================
# 5. CONSTITUENCY TYPE
# =====================================================
print("Step 6: Cleaning Constituency_Type...")

if "constituency_type" in df.columns:
    df["constituency_type"] = df["constituency_type"].replace("GENERAL", "GEN")

# =====================================================
# 6. DEPOSIT LOST
# =====================================================
print("Step 7: Calculating deposit_lost...")

if "votes" in df.columns and "valid_votes" in df.columns:
    df["votes"] = pd.to_numeric(df["votes"], errors="coerce")
    df["valid_votes"] = pd.to_numeric(df["valid_votes"], errors="coerce")

    cond_dep = (
        df["valid_votes"].notna() &
        df["votes"].notna() &
        (df["votes"] < df["valid_votes"] / 6)
    )
    df["deposit_lost"] = cond_dep.map({True: "yes", False: "no"})
else:
    df["deposit_lost"] = "no"

# =====================================================
# 7. MARGIN & MARGIN_PERCENTAGE
# =====================================================
print("Step 8: Calculating Margin and Margin_Percentage...")

# initialise with 0
df["margin"] = 0
df["margin_percentage"] = 0.0

# --- case-insensitive column map ---
colmap = {c.lower(): c for c in df.columns}

votes_col = colmap.get("votes")
const_col = colmap.get("constituency_no")
pos_col   = colmap.get("position")
vsp_col   = colmap.get("vote_share_percentage")  # adjust here if your name differs

# 1) Margin in votes: difference with next candidate by position within each constituency
if votes_col and const_col and pos_col:
    tmp = df.sort_values([const_col, pos_col])
    next_votes = tmp.groupby(const_col)[votes_col].shift(-1)
    df["__next_votes"] = next_votes

    df["margin"] = (df[votes_col] - df["__next_votes"]).where(df["__next_votes"].notna(), 0)
    df["margin"] = df["margin"].astype(int)

# 2) Margin in vote share percentage: same logic, using vote_share_percentage
if vsp_col and const_col and pos_col:
    tmp = df.sort_values([const_col, pos_col])
    next_vsp = tmp.groupby(const_col)[vsp_col].shift(-1)
    df["__next_vsp"] = next_vsp

    df["margin_percentage"] = (
        df[vsp_col] - df["__next_vsp"]
    ).where(df["__next_vsp"].notna(), 0).round(2)

# clean helper columns
df.drop(columns=["__next_votes", "__next_vsp"], inplace=True, errors="ignore")

# =====================================================
# 8. NORMALISE Sex & State_Name
# =====================================================
print("Step 9: Normalising Sex and State_Name...")

if "sex" in df.columns:
    df["sex"] = df["sex"].astype(str).str.upper().str.strip()
    df["sex"] = df["sex"].replace({
        "MALE": "M",
        "FEMALE": "F",
        "THIRD": "O",
        "OTHERS": "O",
        "OTHER": "O"
    })
    df["sex"] = df["sex"].replace({"NAN": "", "NA": ""})

if "state_name" in df.columns:
    df["state_name"] = df["state_name"].astype(str).str.replace(" ", "_").str.upper()

# =====================================================
# 9. COLUMN ORDER
# =====================================================
print("Step 10: Reordering columns...")

main_cols = [
    "delimid", "state_name", "assembly_no", "year", "month", "poll_no", "election_type",
    "constituency_no", "constituency_name", "constituency_type", "electors", "valid_votes", "n_cand",
    "position", "candidate", "sex", "age", "category", "party",
    "votes", "vote_share_percentage", "electors_share_percentage",
    "margin", "margin_percentage", "deposit_lost"
]

existing_main = [c for c in main_cols if c in df.columns]
other_cols = [c for c in df.columns if c not in existing_main]
df = df[existing_main + other_cols]

# =====================================================
# 10. WRITE OUTPUT
# =====================================================
print(f"\nStep 11: Writing {len(df)} rows to '{CLEANED_ELECTION_CSV}'...")
df.to_csv(CLEANED_ELECTION_CSV, index=False, quoting=csv.QUOTE_ALL)
print("\n✅ Done: metadata, N_Cand, Margin, Margin_Percentage, deposit_lost updated.")

In [ ]:
### CODE 4 ####
# - Compare "constituency_map_json" and "election_csv" 
#   on: number_of_rows, page_number_start, page_number_end

import os
import json
import pandas as pd

# ======= CONFIGURE YOUR PATHS =======
print("Loading JSON and CSV...")

# ======= LOAD JSON (CONSTITUENCY_MAP_JSON) =======
with open(CONSTITUENCY_MAP_JSON, "r", encoding="utf-8") as f:
    raw_json = json.load(f)

if isinstance(raw_json, list):
    json_df = pd.DataFrame(raw_json)
elif isinstance(raw_json, dict):
    if "data" in raw_json and isinstance(raw_json["data"], list):
        json_df = pd.DataFrame(raw_json["data"])
    else:
        json_df = pd.DataFrame(list(raw_json.values()))
else:
    raise ValueError("Unexpected JSON structure")

# Required columns in JSON
json_required_cols = [
    "constituency_no",
    "number_of_rows",
    "page_number_start",
    "page_number_end"
]
missing_json = [c for c in json_required_cols if c not in json_df.columns]
if missing_json:
    raise KeyError(f"JSON is missing required columns: {missing_json}")

# Keep only what we need and normalise types
json_df = json_df[json_required_cols].copy()

for col in ["constituency_no", "number_of_rows", "page_number_start", "page_number_end"]:
    json_df[col] = pd.to_numeric(json_df[col], errors="coerce")

json_df = json_df.dropna(subset=["constituency_no"])
json_df["constituency_no"] = json_df["constituency_no"].astype(int)

json_counts = (
    json_df
    .groupby("constituency_no", as_index=False)
    .agg({
        "number_of_rows": "max",          # expected number of rows
        "page_number_start": "min",       # first page where it appears
        "page_number_end": "max"          # last page where it appears
    })
    .rename(columns={
        "number_of_rows": "number_of_rows_json",
        "page_number_start": "page_number_start_json",
        "page_number_end": "page_number_end_json"
    })
)

# ======= LOAD CSV =======
csv_df = pd.read_csv(CLEANED_ELECTION_CSV)

# Required columns in CSV
csv_required_cols = [
    "constituency_no",
    "number_of_rows",
    "page_number_start",
    "page_number_end"
]
missing_csv = [c for c in csv_required_cols if c not in csv_df.columns]
if missing_csv:
    raise KeyError(f"CSV is missing required columns: {missing_csv}")

for col in ["constituency_no", "number_of_rows", "page_number_start", "page_number_end"]:
    csv_df[col] = pd.to_numeric(csv_df[col], errors="coerce")

csv_df = csv_df.dropna(subset=["constituency_no"])
csv_df["constituency_no"] = csv_df["constituency_no"].astype(int)

csv_counts = (
    csv_df
    .groupby("constituency_no", as_index=False)
    .agg({
        "number_of_rows": "max",
        "page_number_start": "min",
        "page_number_end": "max"
    })
    .rename(columns={
        "number_of_rows": "number_of_rows_csv",
        "page_number_start": "page_number_start_csv",
        "page_number_end": "page_number_end_csv"
    })
)

# ======= MERGE & COMPARE =======
merged = json_counts.merge(
    csv_counts,
    on="constituency_no",
    how="outer"
).sort_values("constituency_no")

# Fill NaNs with 0 for comparison, then cast to int
for col in [
    "number_of_rows_json", "number_of_rows_csv",
    "page_number_start_json", "page_number_start_csv",
    "page_number_end_json", "page_number_end_csv"
]:
    merged[col] = merged[col].fillna(0).astype(int)

# Diffs
merged["diff_rows_csv_minus_json"] = merged["number_of_rows_csv"] - merged["number_of_rows_json"]
merged["diff_page_start_csv_minus_json"] = merged["page_number_start_csv"] - merged["page_number_start_json"]
merged["diff_page_end_csv_minus_json"] = merged["page_number_end_csv"] - merged["page_number_end_json"]

# Boolean flags for readability
merged["rows_mismatch"] = merged["number_of_rows_csv"] != merged["number_of_rows_json"]
merged["page_start_mismatch"] = merged["page_number_start_csv"] != merged["page_number_start_json"]
merged["page_end_mismatch"] = merged["page_number_end_csv"] != merged["page_number_end_json"]

# ======= PRINT ONLY DIFFERENCES =======
diff_df = merged[
    merged["rows_mismatch"] |
    merged["page_start_mismatch"] |
    merged["page_end_mismatch"]
]

if diff_df.empty:
    print("No differences: all constituency_no have matching number_of_rows and page ranges in JSON and CSV.")
else:
    # Optional: choose a column order to make it very clear
    cols_order = [
        "constituency_no",
        "number_of_rows_json", "number_of_rows_csv", "diff_rows_csv_minus_json", "rows_mismatch",
        "page_number_start_json", "page_number_start_csv", "diff_page_start_csv_minus_json", "page_start_mismatch",
        "page_number_end_json", "page_number_end_csv", "diff_page_end_csv_minus_json", "page_end_mismatch"
    ]
    existing_cols = [c for c in cols_order if c in diff_df.columns]
    print(diff_df[existing_cols].to_string(index=False))